In [0]:
import pandas as pd
import os
import cv2
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.callbacks import EarlyStopping

In [25]:
train_data = pd.read_csv('train_data.csv')
val_data = pd.read_csv('val_data.csv')

In [27]:
IMG_RESOLUTION = (224, 224)
MIN_AGE = 20
def my_data_generator_train():
  for _,row in train_data.iterrows():
    path = row['image_path']
    image = cv2.imread(path)
    image_resized = cv2.resize(image, IMG_RESOLUTION)
    image_normalized = image_resized / 255.0
    image_flattened = image_normalized.flatten()
    sample = image_flattened
    parts = path.split('\\')
    label = int(parts[-2]) - MIN_AGE
    yield sample, (label,)

In [28]:
def my_data_generator_val():
  for _, row in val_data.iterrows():
    path = row['image_path']
    image = cv2.imread(path)
    image_resized = cv2.resize(image, IMG_RESOLUTION)
    image_normalized = image_resized / 255.0
    image_flattened = image_normalized.flatten()
    sample = image_flattened
    parts = path.split('\\')
    label = int(parts[-2]) - MIN_AGE
    yield sample, (label,)

In [72]:
for _, row in train_data.iterrows():
    path = row['image_path']
    image = cv2.imread(path)
    image_resized = cv2.resize(image, IMG_RESOLUTION)
    image_normalized = image_resized / 255.0
    image_flattened = image_normalized.flatten()
    sample = image_flattened
    parts = path.split('\\')
    label = int(parts[-2]) - MIN_AGE
    print(f'{label} => {path} => {sample}')

20 => data/20-50/train\40\155988.jpg => [0.38431373 0.58039216 0.72156863 ... 0.16862745 0.22352941 0.18039216]
23 => data/20-50/train\43\178287.jpg => [0.45490196 0.37647059 0.42745098 ... 0.05490196 0.06666667 0.1254902 ]
28 => data/20-50/train\48\167830.jpg => [0.18431373 0.2        0.30980392 ... 0.14509804 0.14117647 0.17647059]
29 => data/20-50/train\49\160178.jpg => [0.03529412 0.00784314 0.         ... 0.45490196 0.57647059 0.82352941]
16 => data/20-50/train\36\176068.jpg => [0.12156863 0.45098039 0.56078431 ... 0.8627451  0.83921569 0.84313725]
17 => data/20-50/train\37\159594.jpg => [0.14901961 0.2745098  0.50588235 ... 0.37254902 0.42352941 0.39215686]
22 => data/20-50/train\42\152645.jpg => [0.04313725 0.03921569 0.0745098  ... 0.9372549  0.85490196 0.91372549]
2 => data/20-50/train\22\134662.jpg => [0.42745098 0.40392157 0.37647059 ... 0.44313725 0.4627451  0.49803922]
11 => data/20-50/train\31\168691.jpg => [0.58431373 0.58823529 0.57254902 ... 0.81960784 0.87058824 0.996

In [29]:
batch_size = 32
x_shape = (IMG_RESOLUTION[0] * IMG_RESOLUTION[1] * 3,)
x_type = tf.float32
y_shape = (1,)
y_type = tf.int32

train_ds = tf.data.Dataset.from_generator(my_data_generator_train, output_signature=(
         tf.TensorSpec(shape=x_shape, dtype=x_type),
         tf.TensorSpec(shape=y_shape, dtype=y_type)))

train_ds = train_ds.shuffle(100)
train_ds = train_ds.batch(batch_size)

val_ds = tf.data.Dataset.from_generator(my_data_generator_val, output_signature=(
         tf.TensorSpec(shape=x_shape, dtype=x_type),
         tf.TensorSpec(shape=y_shape, dtype=y_type)))

val_ds = val_ds.batch(batch_size)

In [92]:
L2_REG = regularizers.l2(0.01)

model = models.Sequential([
    tf.keras.layers.Input(shape=x_shape),
    tf.keras.layers.Dense(16, activation='relu', kernel_regularizer=L2_REG),
    tf.keras.layers.Dense(32, activation='relu', kernel_regularizer=L2_REG),
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=L2_REG),
    tf.keras.layers.Dense(1, activation='relu')
])
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mae'])
early_stopping = EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)
model.fit(train_ds, validation_data=val_ds, epochs=20, callbacks=[early_stopping])


Epoch 1/20
836/836 ━━━━━━━━━━━━━━━━━━━━ 115s 135ms/step - loss: 97.7371 - mae: 8.2210 - val_loss: 86.1788 - val_mae: 7.8807
Epoch 2/20
836/836 ━━━━━━━━━━━━━━━━━━━━ 107s 128ms/step - loss: 87.9515 - mae: 7.8451 - val_loss: 84.6927 - val_mae: 7.7578
Epoch 3/20
836/836 ━━━━━━━━━━━━━━━━━━━━ 106s 127ms/step - loss: 86.5799 - mae: 7.7927 - val_loss: 82.5564 - val_mae: 7.6609
Epoch 4/20
836/836 ━━━━━━━━━━━━━━━━━━━━ 110s 131ms/step - loss: 82.5454 - mae: 7.6250 - val_loss: 79.8041 - val_mae: 7.5698
Epoch 5/20
836/836 ━━━━━━━━━━━━━━━━━━━━ 118s 141ms/step - loss: 80.7703 - mae: 7.5762 - val_loss: 79.3346 - val_mae: 7.5604
Epoch 6/20
836/836 ━━━━━━━━━━━━━━━━━━━━ 107s 128ms/step - loss: 79.7733 - mae: 7.5555 - val_loss: 78.4865 - val_mae: 7.5931
Epoch 7/20
836/836 ━━━━━━━━━━━━━━━━━━━━ 106s 126ms/step - loss: 80.6099 - mae: 7.6036 - val_loss: 78.5240 - val_mae: 7.5756
Epoch 8/20
836/836 ━━━━━━━━━━━━━━━━━━━━ 107s 128ms/step - loss: 80.4257 - mae: 7.6306 - val_loss: 78.6273 - val_mae: 7.5852
Epoch 9/

In [ ]:
model.save('model.keras')